<a href="https://colab.research.google.com/github/eaoliveira/senac-telegram-chatbot/blob/main/Telegram_Chat_Bot_Cultural_com_busca_voo__com_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install groq
!pip install python-telegram-bot
!pip install scikit-learn
!pip install nest_asyncio


In [3]:
# configurando o groq

from google.colab import userdata
GROQ_API_KEY = userdata.get('GROQ_API_KEY')
AVIATION_STACK_API_KEY = userdata.get('AVIATION_STACK_API_KEY')
CLIENT_AMADEUS = userdata.get('CLIENT_AMADEUS')
SECRET_AMADEO = userdata.get('SECRET_AMADEO')
TELEGRAM_BOT = userdata.get('TELEGRAM_BOT')

from groq import Groq

client = Groq(api_key = GROQ_API_KEY)


In [ ]:
import logging
import asyncio
from typing import Any, Dict

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import nest_asyncio
from telegram import Update
from telegram.ext import (
    Application,
    CommandHandler,
    MessageHandler,
    filters,
    ContextTypes
)

import json
import requests
from datetime import datetime, timedelta
import random

# Aplicando nest_asyncio
nest_asyncio.apply()

# Configuração do logger
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    level=logging.INFO
)


# Função para gerar datas aleatórias de partida e chegada
def gerar_datas():
    partida = datetime.now() + timedelta(days=random.randint(1, 90))
    chegada = partida + timedelta(hours=random.randint(5, 15))
    return partida.strftime("%d/%m/%Y %H:%M"), chegada.strftime("%d/%m/%Y %H:%M")

# Lista de destinos com informações culturais e companhias fictícias
destinos = [
    {"nome": "Cusco", "companhia": "Cultura Airlines", "descricao": "Porta de entrada para Machu Picchu, cheia de história Inca.", "tempo_estadia": "3 a 5 dias", "clima": "Moderado a frio"},
    {"nome": "Atenas", "companhia": "History Flights", "descricao": "Berço da civilização ocidental, rica em monumentos históricos.", "tempo_estadia": "4 a 7 dias", "clima": "Quente no verão, ameno no inverno"},
    {"nome": "Cidade do México", "companhia": "Maya Wings", "descricao": "Capital vibrante e cultural do México, cheia de história asteca.", "tempo_estadia": "5 a 7 dias", "clima": "Ameno a quente"},
    {"nome": "Roma", "companhia": "Ancient Travels", "descricao": "Cidade eterna, cheia de sítios arqueológicos e arte renascentista.", "tempo_estadia": "5 a 7 dias", "clima": "Ameno na primavera e outono"},
    {"nome": "Cairo", "companhia": "Explorer Airways", "descricao": "Conhecida pelas pirâmides e história do Egito Antigo.", "tempo_estadia": "3 a 5 dias", "clima": "Quente e seco"},
    {"nome": "Kyoto", "companhia": "Heritage Air", "descricao": "Famosa por seus templos históricos e jardins japoneses.", "tempo_estadia": "4 a 6 dias", "clima": "Ameno, com flores de cerejeira na primavera"},
    {"nome": "Paris", "companhia": "ArtVoyage", "descricao": "Centro da arte, cultura e história francesa.", "tempo_estadia": "5 a 7 dias", "clima": "Frio no inverno, agradável no verão"},
    {"nome": "Istanbul", "companhia": "Tradition Wings", "descricao": "Cidade que une o Oriente e o Ocidente, rica em história e cultura.", "tempo_estadia": "3 a 6 dias", "clima": "Moderado, frio no inverno"},
    {"nome": "Jerusalém", "companhia": "Pilgrim Paths", "descricao": "Cidade sagrada com séculos de história religiosa.", "tempo_estadia": "4 a 6 dias", "clima": "Ameno a quente"},
    {"nome": "Fez", "companhia": "Sahara Sky", "descricao": "Conhecida por suas medinas e herança cultural marroquina.", "tempo_estadia": "3 a 5 dias", "clima": "Quente e seco"},
    {"nome": "Beijing", "companhia": "Dynasty Air", "descricao": "Mistura de história antiga e modernidade, com a Grande Muralha próxima.", "tempo_estadia": "5 a 7 dias", "clima": "Frio no inverno, quente no verão"},
    {"nome": "Luang Prabang", "companhia": "Indochina Air", "descricao": "Cidade Patrimônio da UNESCO, conhecida pelos templos budistas.", "tempo_estadia": "4 a 6 dias", "clima": "Ameno com chuvas sazonais"},
    {"nome": "Petra", "companhia": "Oasis Express", "descricao": "Cidade histórica esculpida em rochas, conhecida como a cidade rosa.", "tempo_estadia": "2 a 4 dias", "clima": "Quente e seco"},
    {"nome": "Cartagena", "companhia": "Latin Spirits", "descricao": "Cidade colonial na costa caribenha, cheia de vida e cor.", "tempo_estadia": "3 a 5 dias", "clima": "Quente e úmido"},
    {"nome": "Machu Picchu", "companhia": "Inka Trails", "descricao": "Antiga cidade Inca nas montanhas dos Andes.", "tempo_estadia": "2 a 4 dias", "clima": "Moderado, com chuvas sazonais"},
    {"nome": "Florence", "companhia": "Renaissance Air", "descricao": "Berço da Renascença italiana, cheia de museus e arquitetura histórica.", "tempo_estadia": "3 a 5 dias", "clima": "Ameno no outono e primavera"},
    {"nome": "Varanasi", "companhia": "Spiritual Sojourn", "descricao": "Cidade sagrada na Índia, conhecida pelos rituais no Rio Ganges.", "tempo_estadia": "3 a 5 dias", "clima": "Quente e úmido"},
    {"nome": "Marrakech", "companhia": "Atlas Wings", "descricao": "Cidade vibrante no Marrocos, com mercados, palácios e jardins.", "tempo_estadia": "3 a 5 dias", "clima": "Quente e seco"},
    {"nome": "Lhasa", "companhia": "Tibet Trails", "descricao": "Centro espiritual do Tibete, com templos e monastérios antigos.", "tempo_estadia": "4 a 6 dias", "clima": "Frio e seco, especialmente no inverno"},
    {"nome": "Seul", "companhia": "Gyeong Airlines", "descricao": "Capital da Coreia do Sul, moderna e com rica história cultural.", "tempo_estadia": "3 a 5 dias", "clima": "Frio no inverno, quente no verão"},
    {"nome": "Angkor Wat", "companhia": "Temple Air", "descricao": "Conjunto de templos antigos no Camboja, patrimônio da humanidade.", "tempo_estadia": "2 a 4 dias", "clima": "Quente e úmido"}
]

# Cidades de origem
origens = ["São Paulo", "Rio de Janeiro", "Buenos Aires", "Lisboa", "Cidade do México"]

# Gerar base de dados com 1000 voos fictícios
dados = {"voos": []}
for i in range(1000):
    destino_info = random.choice(destinos)
    destino = destino_info["nome"]
    companhia = destino_info["companhia"]
    origem = random.choice(origens)
    numero_voo = f"{companhia[:2].upper()}{random.randint(100, 999)}"
    data_partida, data_chegada = gerar_datas()
    duracao = f"{random.randint(5, 15)}h {random.randint(0, 59)}m"
    portao_embarque = f"{random.choice('ABCDEF')}{random.randint(1, 20)}"
    terminal = str(random.randint(1, 5))
    disponibilidade = random.choice(["Disponível", "Lotado", "Espera"])
    preco = random.randint(500, 7000)  # Preço numérico para facilitar comparação

    # Adicionar voo à base de dados
    dados["voos"].append({
        "numero_voo": numero_voo,
        "companhia": companhia,
        "origem": origem,
        "destino": destino,
        "data_partida": data_partida,
        "data_chegada": data_chegada,
        "duracao": duracao,
        "portao_embarque": portao_embarque,
        "terminal": terminal,
        "disponibilidade": disponibilidade,
        "preco": preco,
        "descricao_destino": destino_info["descricao"],
        "tempo_estadia": destino_info["tempo_estadia"],
        "clima": destino_info["clima"]
    })

def get_tomorrow_date_formatted():
  """Gets tomorrow's date in the format used by generate_response."""
  tomorrow = datetime.now() + timedelta(days=1)
  return tomorrow.strftime("%Y-%m-%d")

class KnowledgeContextLayer:
    def __init__(self):
        self.knowledge_base = dados

    def get_knowledge(self, key: str) -> Any:
        return self.knowledge_base.get(key)

class InputLayer:
    def __init__(self):
        self.intent_examples = {
            "consulta_voo": ["qual é o preço do voo", "detalhes do voo", "horário de partida", "quando chega o voo", "me fale sobre esse voo", "detalhes do voo"],
            "recomendar_voo": ["me recomenda um voo", "me sugira um voo", "me dê outra opção", "tem mais sugestões?", "voo com preço baixo", "voo com preço alto"],
            "informacao_destino": ["me fale sobre", "quero saber mais sobre", "tempo de estadia em", "clima em", "o que tem em"],
            "saudacao": ["olá", "oi", "bom dia", "boa tarde", "boa noite", "tudo bem?", "como vai?"],
            "conversa": ["qual seu nome?", "quem é você?", "o que você faz?", "você é um robô?"]
        }
        self.vectorizer = CountVectorizer().fit(sum(self.intent_examples.values(), []))
        self.preco_baixo_exemplos = ["baixo preço", "mais barato", "com menor preço", "voo mais barato"]
        self.preco_alto_exemplos = ["alto preço", "mais caro", "com maior preço", "voo mais caro"]

    def process_input(self, user_input: str) -> Dict[str, Any]:
        intent = self._extract_intent(user_input)
        preco_intent = self._extract_preco_intent(user_input)
        return {
            "raw_input": user_input,
            "processed_input": user_input.strip().lower(),
            "tokens": user_input.split(),
            "intent": intent,
            "preco_intent": preco_intent
        }

    def _extract_intent(self, user_input: str) -> str:
        user_vector = self.vectorizer.transform([user_input])
        max_similarity = 0
        best_intent = "consulta_voo"
        for intent, examples in self.intent_examples.items():
            example_vectors = self.vectorizer.transform(examples)
            similarity = cosine_similarity(user_vector, example_vectors).mean()
            if similarity > max_similarity:
                max_similarity = similarity
                best_intent = intent
        return best_intent

    def _extract_preco_intent(self, user_input: str) -> str:
        user_vector = self.vectorizer.transform([user_input])

        # Calcula similaridade para termos de preço baixo
        baixo_similarity = cosine_similarity(user_vector, self.vectorizer.transform(self.preco_baixo_exemplos)).mean()

        # Calcula similaridade para termos de preço alto
        alto_similarity = cosine_similarity(user_vector, self.vectorizer.transform(self.preco_alto_exemplos)).mean()

        if baixo_similarity > alto_similarity and baixo_similarity > 0.3:  # Ajuste de limite de similaridade
            return "baixo"
        elif alto_similarity > baixo_similarity and alto_similarity > 0.3:
            return "alto"
        return ""

class ProcessingLayer:
    def __init__(self, knowledge_context_layer):
        self.knowledge_context_layer = knowledge_context_layer
        self.recommendation_history = []
        self.user_context = ""
        self.conversation_history = []
        self.max_history_length = 5

    def prepare_context(self):
        recent_history = self.conversation_history[-self.max_history_length:]
        context = f"Recent Conversation History: {recent_history}\nUser context: {self.user_context}"
        return context

    def _get_groq_completion(self, query, assistant=None, context=None):
        """
        Sends a query to the Groq API and returns the completion.

        Args:
            query (str): The user's query.
            context (str, optional): The conversation context. Defaults to "".
            assistant (str, optional): A system-level prompt. Defaults to None.

        Returns:
            str: The Groq completion response.
        """
        messages = [
            {"role": "user", "content": query}
        ]
        if context:
            messages.append({"role": "system", "content": context})
        if assistant:
            messages.append({"role": "system", "content": assistant})
        if assistant:
            messages.append({"role": "assistant", "content": assistant})

        completion = client.chat.completions.create(
            model="llama3-70b-8192", messages=messages, stop="```")

        return completion.choices[0].message.content



    def generate_response(self, input_data: Dict[str, Any]) -> str:
        intent = input_data["intent"]
        preco_intent = input_data["preco_intent"]
        if intent == "saudacao":
            return self.handle_conversation(input_data["raw_input"])
        elif intent == "consulta_voo":
            return self.handle_voo_request(input_data["raw_input"])
        elif intent == "recomendar_voo":
            return self.handle_recommendation_request(preco_intent)
        elif intent == "informacao_destino":
            return self.handle_destination_info(input_data["raw_input"])
        return self.handle_conversation(input_data["raw_input"])

    def get_amadeus_api_token(self):
      """Retrieves an access token for the Amadeus API."""
      url = "https://test.api.amadeus.com/v1/security/oauth2/token"
      headers = {"Content-Type": "application/x-www-form-urlencoded"}
      data = {
          "grant_type": "client_credentials",
          "client_id": CLIENT_AMADEUS,
          "client_secret": SECRET_AMADEO
      }
      response = requests.post(url, headers=headers, data=data)
      return response.json()['access_token']

    def fetch_flight_offers(self,iata_code, access_token, date):
      """Fetches flight offers from the Amadeus API."""
      headers = {"Authorization": f"Bearer {access_token}"}
      params = {
          "originLocationCode": "GRU",
          "destinationLocationCode": iata_code,
          "departureDate": date or get_tomorrow_date_formatted(),
          "adults": 1,
          "currencyCode": "BRL",
          "adults": 1,
          "max": 2
      }
      api_result = requests.get('https://test.api.amadeus.com/v2/shopping/flight-offers', headers=headers, params=params)
      return api_result.json()

    def extract_iata_code(self, query):
      """Extracts the IATA code from the query using Groq API."""
      query = f"Extraia o iata code do principal aeroporto e a data da viagem no formato yyyy-mm-dd da pergunta como iata_code e date_travel como um JSON object. Se for mencionada uma data sem especificar o ano, e essa data já tiver passado no ano atual, considere que ela pertence ao ano seguinte ao atual {datetime.now()}. Retorne as informações ajustadas dessa forma: {query} "
      response = self._get_groq_completion(query, "```json")
      return json.loads(response)


    def handle_voo_request(self, query):
        iata_code_dict = self.extract_iata_code(query)
        iata_code =  iata_code_dict['iata_code']
        date_travel = iata_code_dict['date_travel']
        if not iata_code:
            return "Desculpe, não encontrei aeroporto na localizacao mencionada."

        access_token =  self.get_amadeus_api_token()
        api_response =  self.fetch_flight_offers(iata_code, access_token, date_travel)

        if not api_response:
            return "Desculpe, não encontrei informações sobre o voo solicitado."

        context = self.prepare_context()
        query = f"Responda que está enviando os valores e descreva com um formato adequado para telegram sobre os voos listados com Detalhes do voo,companhia (nome completo), Origem, Destino, Partida (com formato dd/mm/aaaa hh:mm), Chegada (com formato dd/mm/aaaa hh:mm),Duração e Preço (formatado como R$): {api_response} "
        response = self._get_groq_completion(query, None, context)

        return response


    def handle_recommendation_request(self, preco_intent):
        voos = self.knowledge_context_layer.get_knowledge("voos")
        if preco_intent == "baixo":
            voo = min(voos, key=lambda v: v["preco"])
        elif preco_intent == "alto":
            voo = max(voos, key=lambda v: v["preco"])
        else:
            voo = next((v for v in voos if v not in self.recommendation_history), None)

        if voo:
            self.recommendation_history.append(voo)
            return (f"Recomendo o voo {voo['numero_voo']} - {voo['companhia']}:\n"
                    f"Origem: {voo['origem']} | Destino: {voo['destino']}\n"
                    f"Partida: {voo['data_partida']} | Chegada: {voo['data_chegada']}\n"
                    f"Duração: {voo['duracao']} | Preço: R$ {voo['preco']:,.2f}".replace(",", "."))
        else:
            return "Desculpe, não há mais recomendações de voos disponíveis."

    def handle_destination_info(self, query):
        query_lower = query.lower()
        try:
          query = f"Extraia o nome, descricao, tempo_estadia (duracao da estadia ideal na cidade), clima e roteiro com foco em atividades culturais com detalhes sobre cada uma pra quantidade de dias sugerida (como um string unica roteiro_viagem, os dias devem ser ter uma quebra de linha entre eles) da pergunta, responda como um JSON object: {query} "

          response = self._get_groq_completion(query, "```json")
        except Exception as e:
          print(f"Não foi possivel procurar por causa do erro: {e}")

        destino = json.loads(response)

        if not destino:
            return "Desculpe, não encontrei informações sobre o destino solicitado."

        return (f"Informações sobre {destino['nome']}:\n"
                f"Descrição: {destino['descricao']}\n"
                f"Tempo de Estadia: {destino['tempo_estadia']}\n"
                f"Clima: {destino['clima']}\n"
                f"Roteiro: {destino['roteiro_viagem']}"
                )

    def handle_conversation(self, query):
        query_lower = query.lower()
        context = self.prepare_context()
        assistant = "Você é um assistente virtual amigavel e util focado em prover experiencias de turismo e responder perguntas sobre voos chamado SenacTurismoBot, Responda o cliente apenas se for relacionado a viagens e voos e destinos, caso o usuario desvie de assunto, responda que não pode falar sobre esse assunto. Sempre responda em portugues. Utilize o nome do usuário sempre que possivel"
        response = self._get_groq_completion(query_lower, assistant,context)
        return response



class SCI:
    def __init__(self):
        self.knowledge_context_layer = KnowledgeContextLayer()
        self.input_layer = InputLayer()
        self.processing_layer = ProcessingLayer(self.knowledge_context_layer)

    def append_to_history(self, user_input, bot_response):
        """Appends a user input and bot response to the conversation history."""
        self.processing_layer.conversation_history.append({"user": user_input, "bot": bot_response})

    def append_user_data(self, user_data):
      """Appends a user name and data to user context."""
      self.processing_layer.user_context = user_data;

    def process_conversation(self, user_input):
        input_data = self.input_layer.process_input(user_input)
        response = self.processing_layer.generate_response(input_data)
        return response


sci = SCI()

# Função para o comando /start
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await update.message.reply_text(
        "Olá! Eu sou seu assistente de voos. Pergunte sobre destinos ou preços."
    )

# Função para lidar com mensagens de texto
async def handle_message(update: Update, context: ContextTypes.DEFAULT_TYPE):
    print(update)
    sci.append_user_data(f"Dados do usuário: nome {update.message.chat.first_name} sobrenome {update.message.chat.last_name} idioma ({update.message.from_user.language_code})")
    user_input = update.message.text.strip()

    resposta = sci.process_conversation(user_input)

    sci.append_to_history(user_input, resposta)

    await update.message.reply_text(resposta)

# Configuração do bot
async def main():
    TOKEN = TELEGRAM_BOT
    application = Application.builder().token(TOKEN).build()

    application.add_handler(CommandHandler("start", start))
    application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_message))

    print("Bot iniciado...")
    await application.run_polling()

# Executando o bot
if __name__ == "__main__":
    asyncio.run(main())
